In [2]:
import pandas as pd
import numpy as np
import random
import requests
from bs4 import BeautifulSoup
import json
import ast


In [3]:
sample_size=500 #sampling here for the purpose of the user study
rtl='rtl'
ltr='ltr'

In [4]:
languages=['enwiki','arwiki','kowiki','cswiki','viwiki','frwiki',
           'fawiki','ptwiki','ruwiki','trwiki','plwiki','hewiki',
           'svwiki','ukwiki','huwiki','hywiki','srwiki','euwiki',
           'arzwiki','cebwiki','dewiki','bnwiki'] #language editions to consider
rtls={
    'enwiki':ltr,'arwiki':rtl,'kowiki':ltr,'cswiki':ltr,'viwiki':ltr,'frwiki':ltr,
           'fawiki':rtl,'ptwiki':ltr,'ruwiki':ltr,'trwiki':ltr,'plwiki':ltr,'hewiki':rtl,
           'svwiki':ltr,'ukwiki':ltr,'huwiki':ltr,'hywiki':ltr,'srwiki':ltr,'euwiki':ltr,
           'arzwiki':rtl,'cebwiki':ltr,'dewiki':ltr,'bnwiki':ltr
}

Quering SQL replicas to get the list of images in each category through the categorylinks table (unavailable on Hive)

**There must be a better way now that categorylinks is on hive**


The function `get_properties` works as follows:
* I downlaods the html page of the image then extract information such as caption, description, categories and structure data. 
All this is done via the html page - it might make sense to improve the code below and integrate with existing methods that extract this information.

In [5]:
def get_properties(image_name,language,rtl):
    #
    #getting html page content or return 404 if not available
    try:
        urll = 'https://commons.wikimedia.org/wiki/File:'+image_name.replace(' ','_')+'?uselang='+language
    #print(urll)
        page = requests.get(urll)
        if page.status_code == 404:
            return None,None,None,None,None,None,None,None,None
    except:
            return None,None,None,None,None,None,None,None,None
    soup = BeautifulSoup(page.content, 'html.parser')
    #
    #getting descriptions
    desc = None
    descriptions = []
    descriptions=soup.find_all('div', class_='description mw-content-'+rtl+' '+language)
    if len (descriptions)==0:
        descriptions=soup.find_all(lambda tag: tag.name == ['td'] and 
                                tag.get('class') == ['description'] and tag.get('lang') == [language])
        if len (descriptions)==0:
            descriptions=soup.find_all(lambda tag: tag.name == ['div'] and 
                                    tag.get('class') == ['description'] and tag.get('lang') == [language])
    if len (descriptions)==0 and language =='en': 
        descriptions=soup.find_all(lambda tag: tag.name == 'div' and 
                               tag.get('class') == ['description'])
        if len (descriptions)==0:
            descriptions=soup.find_all(lambda tag: tag.name == 'td' and 
                               tag.get('class') == ['description'])
        if len(descriptions)>0 and len(descriptions[0].find_all("div"))>0:
            descriptions=[]
    if len(descriptions)>0:
        for description in descriptions:
            desc = description.get_text().strip()
#     for english (there are 3 different ways to get descriptions)
    #
    #getting captions
    capt = None
    try:
        caption = soup.find_all('div', class_ = 'wbmi-caption-value', lang=language)[0]
        if (not caption is None and not len(caption['class'])>1):
                capt = caption.get_text()
    except IndexError:
        pass
    #
#     #getting categories
#     categories = soup.find_all('div')
#     for category in categories:
#         c = category.find(href='/wiki/Special:Categories')
#         c2 = category.find('div')
#         if (c != None and c2 == None): #get the "smallest" div around the specific href
#             cats= category.find_all(text=True)[2:]
#             break
#         else:
#             cats=[]
#     #
#     #parsing image atrributes to get size, user who uploaded the image, and date
#     size=soup.find('img', attrs={'data-file-width': True})
#     size=str(size['data-file-height'])+'x'+str(size['data-file-width'])
#     user=soup.find(class_='mw-userlink')
#     user=user['title']
#     date=soup.find('td',class_='filehistory-selected')
#     date=date.get_text()
#     #
    #parsing the structured data json to get tags and copuright
    depicts_statements = soup.find_all('div', attrs={'data-formatvalue': True,'data-property':['P180','P275']})
    depicts=[]
    copyright=None
    for depiction in depicts_statements:
            data=json.loads(depiction['data-formatvalue'])
            if len(data)>0:
                for d in list(data.values())[1:]:
                    extracted=d['text/plain'][language]
                    propname=(list(extracted.keys()))[0]
                    if propname=='P180':
                        #print(extracted)
                        depicts.append(list(extracted.values())[0])
                    elif propname=='P275':
                        copyright=list(extracted.values())[0]
    #
    #parsing image extension
    extension=image_name[image_name.rfind('.')+1:]
    #return desc,capt,cats,size,user,depicts,date,copyright,extension
    return desc,capt,depicts

In [6]:
properties={}

In [ ]:
#properties={}
for wiki in languages:
    properties[wiki]={}
    language=wiki.replace('wiki','')
    rtllan=rtls[wiki]
    #this has to be substituted with the right input method/file extracting unillustrated articles and their corresponding potential candidates 
    data=pd.read_csv('Output/'+wiki+'_wd_image_candidates.tsv',sep='\t')
    data=data.drop('Unnamed: 0',axis=1)
    #for the purpose of this proof of concept
    data_small=data.sample(min(len(data),sample_size))
    #downloading properties for each available image
    print(language)
    for imagelist in data_small['top_candidates']:
        for imagedic in ast.literal_eval(imagelist):
            image=imagedic['image']
            if image is not None:
                    properties[wiki][image]=get_properties(image,language,rtllan)
            else:
                    properties[wiki][image]=[None,None,None]#,None,None,None,None,None,None])
    #updating tabl


en
